# Lab 1 – Deep Learning Backends for Lane Segmentation
---
## Overview

This lab introduces the **deep learning perception backends** used for lane segmentation in the AutoCar-Kit Lane Keeping system.

You will explore how multiple state-of-the-art segmentation models can be integrated as **interchangeable perception modules**, producing consistent binary lane masks for downstream processing.

The output of this lab serves as the **input foundation** for all subsequent stages, including ROI filtering, BEV transformation, lane geometry estimation, and steering control.

---
In the overall project pipeline, the first block is:

> **Deep Learning Backend Segmentation** (YOLOv8 / PIDNet / TwinLiteNet / BiSeNetV2)
> → generates **binary lane mask `mask01`**

Lab 1 focuses on the backend:

- Correctly initialize the backend lane segmentation.

- Call `infer_mask01(frame_bgr)` to generate `mask01`.

- Visualize the segmentation results.

- Save `Lab1_mask01_*.png` for 10 frames (reusable in Lab 2).

> This is the **student** version: important code sections are hidden using `...` + general hints. You need to read the original code in the `AI/` folder yourself to complete the process.

# Code Reading Suggestions:

- Find the `build_lane()` function in `AI/main.py` to see how the project initializes the backend.

- View the configuration parameters of each model in `AI/configs/config.py`.

- In Tasks 4–5–6, the infer method you need to use is the same method

that the project is using to generate `mask01` before passing it to the ROI.

## Learning Objectives

- Understand the 4-backend lane segmentation structure of the project.

- Initialize a backend from `model_name`.

- Use `infer_mask01` to create a binary lane mask.

- Visually compare the results of each backend.

- Prepare the `Lab1_mask01_*.png` file for Lab 2.

# 0. Preparing the environment


In [ ]:
# 0.1. Importing basic libraries
import sys
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
# 0.2. Set the path to the AI ​​folder and import the configuration + backend

ROOT = Path("../..").resolve()
AI_ROOT = ROOT / "AI"
sys.path.append(str(AI_ROOT))

print("Project ROOT:", ROOT)
print("AI ROOT     :", AI_ROOT)

from configs import config as C

from LaneDetection.backends.yolov8_backend import YoloV8Backend
from LaneDetection.backends.pidnet_backend import PIDNetBackend
from LaneDetection.backends.twinlite_backend import TwinLiteBackend
from LaneDetection.backends.bisenetv2_backend import BiseNetV2Backend

print("LANE_MODEL  (default):", C.LANE_MODEL)
print("LANE_WEIGHTS(default):", C.LANE_WEIGHTS)


# If the machine only has a CPU, run the cell below to force the CPU to be used.

In [ ]:
import torch

print("Before override, C.DEVICE =", C.DEVICE)
print("torch.cuda.is_available():", torch.cuda.is_available())
C.DEVICE = "cpu"

print("After override,  C.DEVICE =", C.DEVICE)


## 1. Build a deep learning backend

### Overview

In the project's real pipeline, the backend module acts as the "perception brain": it receives raw images from the camera and returns a binary mask lane.

The AI/main.py file shows you how the system actually creates the backend corresponding to the selected model (YOLOv8 / PIDNet / TwinLite / BiSeNet).

All backends are built from configuration and weights:

- Configuration contains technical specifications → input size, number of classes, threshold, PIDNet/TwinLite architecture.

- Weights contain the learned parameters of the model.

When writing build_backend(model_name) in this Lab, you are simulating the backend initialization process of a real system.

### To complete, you need to understand 3 things:

**1) AI/main.py → How the real system creates the backend**

Find the build_lane() function to see:

- It checks `LANE_MODEL`

- It maps the model → to the correct backend class (YoloV8Backend, PIDNetBackend, etc.)

- It passes parameters from the configuration to the class

> ➡ This is the logic you need to recreate in the lab.

**2) AI/configs/config.py → Find model parameters**

Here you will find:

- `PIDNET_H`, `PIDNET_W`, `PIDNET_THR`, `PIDNET_ARCH`

- `TWIN_H`, `TWIN_W`, `TWIN_THR`, `TWIN_NUM_CLASSES`

- `IMGSZ`, `CONF` for YOLOv8

- `BISENET_*` for BiSeNet

>➡ These are the “materials” you must pass in when initializing the backend.

**3) AI/LaneDetection/Lane_weight/ → Corresponding weight file**

Check this folder to find:

- which weight file each model uses

- the correct folder name

- the correct extension (.pt or .pth).

>➡ You are accurately simulating how a real pipeline finds the weight file upon startup.

In [ ]:
# 1.1. Dictionary: model name -> corresponding weight file name in the Lane_weight folder
LANE_WEIGHT_NAME = {
    "yolov8":  "Yolo_v8/best.pt",
    "pidnet":  "PIDNet/best.pt",
    "twinlite":"TwinLite/best.pth",
    "bisenet": "BiseNet/best.pth",
}

def get_lane_weight_path(model_name: str) -> Path:
    """
    Returns the full path to the weight file for model_name.

    General hints:
    - The root directory containing the weights is located in the LaneDetection/Lane_weight folder of your project.
    - You can get the specific filename from the LANE_WEIGHT_NAME dictionary above.
    """
    name = model_name.lower()
    if name not in LANE_WEIGHT_NAME:
        raise ValueError(f"Unknown lane model: {model_name}")

    # TODO: Concatenate the weight folder path with the corresponding filename.
    # You need to decide which elements to use from the C.ROOT variable.
    weights_dir = ...   # Example: Path to the folder containing all weight files
    weight_rel  = ...   # Example: File name of weight taken from the dictionary LANE_WEIGHT_NAME
    weight_path = ...   # Combine folder + file name to get the full path

    return weight_path


In [ ]:
def build_backend(model_name: str):
    """
    Initialize the corresponding backend lane segmentation.

    Parameters
    ----------
    model_name : str
        'yolov8' | 'pidnet' | 'twinlite' | 'bisenet'
    """
    name = model_name.lower()
    weights = get_lane_weight_path(name)

    # TODO: Complete each if/elif branch.
    # General suggestions:
    # - Configuration parameters (input size, threshold, number of classes, etc.) are defined in the project's config file.
    # - Open configs/config.py to see the names of the parameters corresponding to each model.
    if name == "yolov8":
        backend = YoloV8Backend(
            weights=str(weights),
            device=C.DEVICE,
            imgsz=...,   # Hint: Input image sizes are shared across YOLO
            conf=...,    # Hint: YOLO confidence threshold
        )
    elif name == "pidnet":
        backend = PIDNetBackend(
            weights=str(weights),
            device=C.DEVICE,
            input_h=...,   # Hint: PIDNet input height
            input_w=...,   # Hint: PIDNet input width
            thr=...,       # Hint: PIDNet segmentation threshold
            arch=...,      # Suggestion: Choose PIDNet architecture
        )
    elif name == "twinlite":
        backend = TwinLiteBackend(
            weights=str(weights),
            device=C.DEVICE,
            input_h=...,          # Hint: TwinLite input height
            input_w=...,          # Hint: TwinLite input width
            thr=...,              # Hint: TwinLite segmentation threshold
            num_classes=...,      # Hint: TwinLite output class number
        )
    elif name == "bisenet":
        backend = BiseNetV2Backend(
            weights=str(weights),
            device=C.DEVICE,
            input_h=...,          # Hint: input height of BiSeNetV2
            input_w=...,          # Hint: input width of BiSeNetV2
            num_classes=...,      # Hint: number of output classes of BiSeNetV2
        )
    else:
        raise ValueError(f"Unknown model_name: {model_name}")

    return backend


In [ ]:
# 1.2. After filling in TODO, run this cell for a quick check.
for name in ["yolov8", "pidnet", "twinlite", "bisenet"]:
    try:
        b = build_backend(name)
        print(f"{name:8s} ->", b.__class__.__name__)
    except Exception as e:
        print(f"{name:8s} -> ERROR:", e)


After reading the files above, you will understand that the process from model_name → finding weights → getting configurations → creating backend is a standard procedure, similar to "selecting perception modules" in a real autonomous vehicle.

## 2. Load sample frames


### Overview

Before loading images into the backend, you must ensure that the images are loaded correctly, as every subsequent step depends on the quality of the input images.

This task helps you understand how the pipeline actually receives frames from the camera and processes each frame.

### To complete:

**1) Understand the correct frame directory structure**

You must accurately identify:

- Where the folder containing the frames is located

- What the pattern file is (*.jpg, *.png, …)

- What the file name format is (Lab1_frames_01.png, frame_001.jpg…)

>➡ Because glob() will not find the frame if the pattern is incorrect → number of frames = 0.

**2) Use OpenCV to read the images**

cv2.imread() reads the image in BGR format, which is the format required by the backend.

You need to check:

- whether the image loads correctly

- whether the image shape is correct (H, W, 3)

- Similar to a real pipeline, checking the first 1–2 frames helps ensure valid camera input.

In [ ]:
FRAMES_DIR = ROOT / "Lab" / "Lab1" / "Lab1_frames"

frame_paths = sorted(FRAMES_DIR.glob("frame_*.jpg"))
print("Number of frames found:", len(frame_paths))
for p in frame_paths[:5]:
    print(" -", p)


In [ ]:
# View a frame
if len(frame_paths) == 0:
    raise RuntimeError("No frames found in data/lab1_frames")

sample_path = frame_paths[0]
frame_bgr = cv2.imread(str(sample_path))
if frame_bgr is None:
    raise FileNotFoundError(f"Unable to read the image: {sample_path}")

frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,4))
plt.imshow(frame_rgb)
plt.title(sample_path.name)
plt.axis("off")
plt.show()


After Task 2, you understand how to properly pipeline frames from the camera → this is a fundamental step for the backend to process correctly.

# 3.  Build a function to display frame + mask lane


### Overview

In a real project, the lane mask is overlaid onto the image for debugging and visualization.

Overlaying helps you see:

- How the model segments the lane

- Whether the mask is seamless or noisy

- Whether the lane matches the actual path.

- The visualize function in this Task accurately simulates the overlay logic of a real pipeline in main.py or `lane_pipeline.py`.

### To write the show_frame_and_mask function:

**1) Convert the image to RGB**

Matplotlib displays images in RGB → you must convert from BGR to RGB.

**2) Create an overlay layer**

- Copy the original frame

- Fill the mask area (frame[mask>0] = [255,0,0])

- Choose any lane color

**3) Use cv2.addWeighted**

Create a transparent frame, allowing you to see both the lane and the background.

Genuine pipelines use this exact technique.

In [ ]:
def show_frame_and_mask(frame_bgr, mask01, title=""):
    """Display the original frame and the frame with the overlay lane."""
    # TODO: Converting from BGR (OpenCV) to RGB (matplotlib)
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    # TODO: Create a copy and color the lane area (mask01 == 1)
    overlay = frame_rgb.copy()
    overlay[mask01 > 0] = ...   # Hint: Use a striking color like pure red

    # TODO: Mix frame_rgb and overlay using addWeighted to create a transparent effect.
    alpha = 0.5
    blended = cv2.addWeighted(frame_rgb, 1 - alpha, overlay, alpha, 0)

    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.imshow(frame_rgb)
    plt.title(" Original frame")
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.imshow(blended)
    plt.title(title)
    plt.axis("off")

    plt.tight_layout()
    plt.show()


You understand the visualization mechanism of pipeline lane detection and know how to overlay masks correctly, just like in a real project.

# 4. Test run a backend (TwinliteNet)


### Overview

Each backend has a different infer method: resizing input, normalizing, changing the order of color channels.

However, the backend class has "wrapped" everything into one function:

`backend.infer_mask01(frame_bgr)`

The actual pipeline only calls this function.

Task 4 helps you see how a backend works independently.

### To complete this task you need:

**1) Identify the infer function in the backend file**

Examples:

- YOLOv8: infer_mask01()

- PIDNet: infer_mask01()

- TwinLite: infer_mask01()

- BiSeNet: infer_mask01()

They all return a mask.

**2) See the required input**

Backends usually receive the original image BGR (resized inside the class).

**3) View the output mask**

The output can be:

- 0–255

- or 0–1

But you should always cast it to 0–1 in the lab for consistency:

`(mask > 0).astype(np.uint8)`

In [ ]:
backend = build_backend("twinlite") # 🔁 Change the model name if you want to use a different bankend (Yolov8, BiseNet, PidNet)

sample_path = frame_paths[0]
frame_bgr = cv2.imread(str(sample_path))

# TODO: Call the backend's infer function to get the lane segment mask.
mask = ...      # Hint: Use a backend method with the input frame_bgr

# TODO: Convert mask to mask01 in 0/1 format (True/False -> 1/0)
mask01 = ...    

print("Frame shape:", frame_bgr.shape)
print("Mask  shape:", getattr(mask, "shape", None))

show_frame_and_mask(frame_bgr, mask01, title="twinlite – mask01") # 🔁


You understand that the backend is the “perception black box”: just input the image → remove the mask.
This is the exact logic that a real pipeline uses.

# 5. Comparing 4 backends on the same frame


### Overview

This task simulates a real-world model benchmarking problem:
- Using the same frame → running through 4 different models → comparing quality and stability.

- The research team's actual pipeline is usually exactly the same.

### To complete:

**1) Reuse code from Task 3 & Task 4**

- Task 3: reuse the overlay function

- Task 4: use build_backend(name) and infer_mask01(...)

**2) Loop through the list of models**

["yolov8", "pidnet", "twinlite", "bisenet"]

**3) Visualize each backend**

Give clear titles: YOLOv8 / PIDNet / TwinLite / BiSeNet.

In [ ]:
model_names = ["yolov8", "pidnet", "twinlite", "bisenet"]

sample_path = frame_paths[0]
frame_bgr = cv2.imread(str(sample_path))

plt.figure(figsize=(12, 6))

for i, name in enumerate(model_names, start=1):
    # TODO: initialize the corresponding backend.
    backend = ...

    # TODO: Infer mask and switch to mask01 0/1
    mask = ...
    mask01 = ...

    # TODO: Create an overlay from mask01 (the logic can be reused in Task 3)
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    overlay = frame_rgb.copy()
    overlay[mask01 > 0] = [255, 0, 0]
    blended = cv2.addWeighted(frame_rgb, 0.5, overlay, 0.5, 0)

    plt.subplot(2, 2, i)
    plt.imshow(blended)
    plt.title(name.upper())
    plt.axis("off")

plt.tight_layout()
plt.show()


You understand the strengths and weaknesses of each model and see why the pipeline of an autonomous vehicle needs a strong backend.

# 6. Create and store binary lane masks for analysis


### Overview

In the entire project's pipeline lane-keeping, mask01 (binary mask) is a required input for the following steps:

- ROI selection

- Morphology

- BEV transform

- Lane geometry estimation

- Controller (EMA + steering output)

Task 6 helps you create a standard mask01 set to reuse exactly like the real pipeline.

### To complete this Task:

**1) Loop through all frames**

Infer mask → cast to 0–1 → save to list.

**2) Stack into a numpy array**

`all_masks = np.stack(all_masks, axis=0)`

The shape will be: `(10, H, W)`

3) Save using numpy

For .py files:

`np.save("lab1_mask01.npy", all_masks)`

Or PNG (your version)

- mask01 → 0–1

- multiply 255 → PNG

- save each file

In [ ]:

# MODEL_FOR_LAB2 = "twinlite"  # or 'pidnet' / 'yolov8' / 'bisenet'
MODEL_FOR_LAB2 = "twinlite" #🔁 select bankend (Yolov8, BiseNet, PidNet)

backend = build_backend(MODEL_FOR_LAB2)

# TODO: Create a folder to save the PNG output.
OUT_DIR = ROOT / "Lab" / "Lab1" / "Lab1_masks"
OUT_DIR.mkdir(parents=True, exist_ok=True)

for i, p in enumerate(frame_paths, start=1):
    frame_bgr = cv2.imread(str(p))
    if frame_bgr is None:
        raise FileNotFoundError(f"Unable to read the image: {p}")

    # TODO: Infer mask from backend
    mask = ...

    # TODO: Convert mask to binary 0–1 (uint8)
    mask01 = ...

    # TODO: Multiply by 255 to save the PNG image (0–1 → 0–255)
    mask_png = ...

    out_path = OUT_DIR / f"Lab1_mask01_{i:02d}.png"

    cv2.imwrite(str(out_path), mask_png)

print("➡ I've finished generating the PNG mask for 10 frames!", out_path)


You have completed the perception part of the pipeline lane detection.
From here, Lab 2 & Lab 3 will use this mask to build ROI → BEV → Control as in the real project.

# 7. Summary

Upon completing all the TODOs, you will have:

- Initialized the backend corresponding to each model by reading and understanding the project's configuration file.

- Used the backend's infer function to create a binary lane mask.

- Visually compared four backends on the same frame.

- Generated the `Lab1_mask01_*.png` file for Lab 2.

👉 Don't forget to save the `.png` files in the `/Lab/Lab1_mask/Lab1_mask01_01.png ... Lab1_mask01_10` directory.